In [1]:
import numpy as np # linear algebra
import pandas as pd #

In [2]:
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])
member_info=member_info[['uid','topic_n']]
from tqdm import tqdm
tqdm.pandas()

In [3]:
import pickle
def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)
topic_w2v=load_obj('topic_vectors_64d')

In [4]:
def split_(x):
    if x!='-1':
        x=x.split(',')
        topic=[i.split(':')[0] for i in x]
        weight=[float(i.split(':')[1]) for i in x]
        result=np.zeros(64)
        for i in range(len(topic)):
            result+=weight[i]*topic_w2v[topic[i]]['vector']
        return result
    else:return np.NaN

In [ ]:
member_info['weight']=member_info['topic_n'].progress_apply(lambda x:split_(x))

 41%|████      | 788144/1931654 [00:27<00:37, 30507.06it/s]

In [ ]:
member_info.dropna(inplace=True)
member_info=member_info.reset_index(drop=True)

In [ ]:
weight=pd.DataFrame(member_info['weight'].to_list())

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                else:
                    df[col] = df[col].astype(np.float16)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
weight=reduce_mem_usage(weight)

In [ ]:
member_info.drop(['topic_n','weight'],axis=1,inplace=True)

In [ ]:
member_info=pd.concat([member_info,weight],axis=1)

In [13]:
member_info.to_hdf('feature_weight_vector.h5', key='data')

/root/anaconda3/lib/python3.7/site-packages/pandas/io/pytables.py:274: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  f(store)
